In [81]:
from docx import Document
import os
import pandas as pd
import hashlib
import ftfy 
import re
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.metrics import classification_report, confusion_matrix
from scipy.sparse import hstack

In [82]:
# used for fixing mojibike and encoding glitcheces
def clean_text(text):
    text = ftfy.fix_text(text)  
    return text.strip()

# getting itallic parts of the text 
def extract_italic_text(doc_path):
    doc = Document(doc_path)
    italic_texts = []
    for para in doc.paragraphs:
        for run in para.runs:
            if run.italic:
                italic_texts.append(clean_text(run.text))
    return " ".join(italic_texts)

# path to the document where the letters are stored
folder_path = r"C:\Users\Kvrgic\Desktop\Soriano\Soriano_texts2"

# normalizing text
def get_hash(text):
    normalized = text.lower().strip()
    normalized = " ".join(normalized.split())
    return hashlib.md5(normalized.encode("utf-8")).hexdigest()
data_italic = []
seen_hashes = set()

for filename in os.listdir(folder_path):
    if filename.endswith(".docx") and not filename.startswith("~$"):
        file_path = os.path.join(folder_path, filename)
        italic_text = extract_italic_text(file_path)
        if italic_text:
            text_hash = get_hash(italic_text)
            if text_hash not in seen_hashes:
                data_italic.append(italic_text)
                seen_hashes.add(text_hash)

# saving the result to local machine
df = pd.DataFrame(data_italic, columns=["text"])
df.to_csv("italic_texts_dataset_cleaned.csv", index=False, encoding="utf-8-sig")

In [83]:
#print(data_italic)

In [84]:
# loading the cleaned italic text
italic_df = pd.read_csv("italic_texts_dataset_cleaned.csv", encoding="utf-8")
# regex for reported speech
direct_speech_patterns = [
    # core direct speech 
    r"\bmi disse\b", r"\bmi rispose\b", r"\bmi replicò\b", r"\bmi dichiarò\b", r"\bmi aggiunse\b", r"\bmi promise\b",
    r"\bet ella\b.*?\bdisse\b", r"\bet ella\b.*?\brispose\b",
    r"\ba me\b.*?\bdisse\b", r"\ba me\b.*?\brispose\b", r"\ba me\b.*?\bparlò\b",
    r"\busc[iì]\b.*?\ba dirmi\b",

    # gerunds and participial phrases 
    r"\bdicendo\b", r"\bparlando\b", r"\bcontinuando\b", r"\baggiungendo\b", r"\bassicurando\b",
    r"\bsoggiungendo\b", r"\basserendo\b", r"\bdichiarando\b", r"\btestimoniando\b",

    # imperfect indicative verbs
    r"\bmi replicava\b", r"\bmi parlava\b", r"\bmi diceva\b", r"\bmi soggiungeva\b", r"\bmi raccordava\b",

    # past simple
    r"\bdisse\b", r"\brispose\b", r"\bcontinuò\b", r"\baggiunse\b", r"\breplicò\b", r"\bdichiarò\b", r"\bsoggiunse\b", r"\bprofferì\b",

    # first-person verb forms 
    r"\b(io )?disse\b", r"\b(io )?risposi\b", r"\b(io )?aggiunsi\b", r"\b(io )?replicai\b", r"\bio non aggiunsi altro\b",
    r"\bet io andai rispondendo\b", r"\bandai rispondendo\b", r"\bandai portando\b",

    # passive and indirect reporting
    r"\bfu proferito\b", r"\bfu detto\b", r"\bfu risposto\b", r"\bil profferì con qualche sorriso\b",
    r"\bil che mi è sta\[to\] detto anco da altri\b", r"\bet a me lo diceva in confidenza\b",

    # edge continuation or extension
    r"\bet poi soggiunse\b", r"\bet aggiunse\b", r"\bet continuò\b",
    r"\bet io risposi\b", r"\bio replicai\b", r"\bio risposi\b",

    # contextual gerunds
    r"\bvedendo ch’io\b", r"\bet essendo\b", r"\bet essere\b",
    r"\bdoppo haver\b.*?\bdiscorso\b",
]
pattern = re.compile("|".join(direct_speech_patterns), re.IGNORECASE)
# sentence splitter
def simple_sent_tokenize(text):
    parts = re.split(r'(?<=[\.\!\?])\s+', text)
    return [p.strip() for p in parts if p.strip()]

# cleaning the text
def clean_text(text):
    return ftfy.fix_text(text).strip()

# building labels is_reported, previous_reported, next_reported, based on regex
records = []
for frag in italic_df["text"].astype(str):
    cleaned_frag = clean_text(frag)
    sentences = simple_sent_tokenize(cleaned_frag)
    prev1 = False
    prev2 = False
    temp_records = []
    for sent in sentences:
        fixed_sent = clean_text(sent)
        fixed_sent_lower = fixed_sent.lower()
        is_reported = bool(pattern.search(fixed_sent_lower))
        
        # gratie special rule
        if re.search(r"\bgratie\b.*?(etc)?", fixed_sent_lower) and len(temp_records) > 0:
            is_reported = False
            temp_records[-1]["is_reported"] = True
            
        prev_reported = prev1 or prev2
        temp_records.append({"sentence": fixed_sent,"is_reported": is_reported,"previous_reported": prev_reported})
        prev2 = prev1
        prev1 = is_reported

    # next_reported
    for i in range(len(temp_records)):
        next1 = temp_records[i + 1]["is_reported"] if i + 1 < len(temp_records) else False
        temp_records[i]["next_reported"] = next1
    records.extend(temp_records)

# saving of documents on local machine
sent_df = pd.DataFrame(records)
sent_df.to_csv("italic_sentences_labeled.csv", index=False, encoding="utf-8-sig")

In [87]:
# fixing document labeledCORRECTED
rows = []
with open("italic_sentences_labeledCORRECTED.csv", encoding="utf-8-sig") as f:
    for line in f:
        parts = [p.strip() for p in line.strip().split(",")]
        if len(parts) >= 3:
            sentence = ",".join(parts[:-2])
            is_reported = parts[-2]
            previous_reported = parts[-1]
            rows.append({"sentence": sentence,"is_reported": is_reported,"previous_reported": previous_reported})
            
df = pd.DataFrame(rows)
df = df[df["is_reported"] != "is_reported"]

# cleaning pevious_reported
df["previous_reported"] = (df["previous_reported"].astype(str).str.strip(" ;").map({"True": 1, "False": 0}))
# cleaning is_reported
df["is_reported"] = (df["is_reported"].astype(str).str.strip(' ;').map({"True": 1, "False": 0}))
df["previous_reported"] = df["previous_reported"].fillna(0).astype(int)
print(df["previous_reported"].unique())
print(df["is_reported"].value_counts())

# computing previous_reported using positional indexing
prev_reported = []
for i in range(len(df)):
    prev1 = df.iloc[i - 1]["is_reported"] if i - 1 >= 0 else False
    prev2 = df.iloc[i - 2]["is_reported"] if i - 2 >= 0 else False
    prev_reported.append(prev1 or prev2)

df["previous_reported"] = prev_reported
df["is_reported"] = df["is_reported"].map({1: "True", 0: "False"})
df["previous_reported"] = df["previous_reported"].map({1: "True", 0: "False"})

# saving to local machine
df.to_csv("italic_sentences_labeledCORRECTED2.csv", index=False, encoding="utf-8-sig")

[0 1]
is_reported
0    496
1    199
Name: count, dtype: int64


In [88]:
# adding next_reported to labeledCORRECTED2
# loading corrected labeled file
df = pd.read_csv("italic_sentences_labeledCORRECTED2.csv", encoding="utf-8-sig", quotechar='"')
df["is_reported"] = df["is_reported"].astype(str).str.strip().map({"True": True, "False": False})
df["previous_reported"] = df["previous_reported"].astype(str).str.strip().map({"True": True, "False": False})

# making next_reported
next_reported = df["is_reported"].shift(-1).fillna(False)
df["next_reported"] = next_reported.astype(bool)

# saving to local machine
df.to_csv("italic_sentences_labeledCORRECTED3.csv", index=False, encoding="utf-8-sig", quotechar='"')

C:\Users\Kvrgic\AppData\Local\Temp\ipykernel_22056\1834454222.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  next_reported = df["is_reported"].shift(-1).fillna(False)


In [89]:
# defining x and y
X_text = df["sentence"]
X_prev = df["previous_reported"].astype(int).values.reshape(-1, 1)
X_next = df["next_reported"].astype(int).values.reshape(-1, 1)
y = df["is_reported"].astype(int)

# train and split
X_text_train, X_text_test, X_prev_train, X_prev_test, X_next_train, X_next_test, y_train, y_test = train_test_split(X_text, X_prev, X_next, y, test_size=0.2, random_state=42, stratify=y)

# TF-IDF vectorization
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
X_text_vec = vectorizer.fit_transform(X_text)
X_text_train_vec = vectorizer.transform(X_text_train)
X_text_test_vec = vectorizer.transform(X_text_test)

X_combined = hstack([X_text_vec, X_prev, X_next])
X_train_combined = hstack([X_text_train_vec, X_prev_train, X_next_train])
X_test_combined = hstack([X_text_test_vec, X_prev_test, X_next_test])

# cross validation
model = LogisticRegression(max_iter=1000, class_weight="balanced")
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = ["accuracy", "precision", "recall", "f1"]
cv_results = cross_validate(model, X_combined, y, cv=cv, scoring=scoring)

print("Cross validation results")
for score in scoring:
    print(f"{score}: {cv_results['test_' + score].mean():.3f}")

# training and evaluation
model.fit(X_train_combined, y_train)
y_pred = model.predict(X_test_combined)

print("Test results")
print(classification_report(y_test, y_pred))

Cross validation results
accuracy: 0.791
precision: 0.627
recall: 0.698
f1: 0.659
Test results
              precision    recall  f1-score   support

           0       0.86      0.87      0.86        99
           1       0.67      0.65      0.66        40

    accuracy                           0.81       139
   macro avg       0.76      0.76      0.76       139
weighted avg       0.80      0.81      0.81       139

